In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

In [0]:
df_faturas = (
    spark.read.format("delta").load("/Volumes/workspace/default/raw/tb_faturas")
    .select(
        F.col("fatura_id"),
        F.col("reserva_id"),
        F.col("data_cobranca"),
        F.round(F.col("valor_total").cast("double"), 2).alias("valor_total"),
        F.col("impostos"),
        F.col("forma_pagamento"),
        F.col("status_pagamento")
    )
    .withColumn("data_cobranca", F.to_date(F.col("data_cobranca")))
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast("date"))
)

display(df_faturas.limit(5))


In [0]:
catalog_path = "/Volumes/workspace/default/trusted/tb_faturas"

if DeltaTable.isDeltaTable(spark, catalog_path):
    delta_table = DeltaTable.forPath(spark, catalog_path)
    
    # Merge (upsert) com atualização da update_date
    delta_table.alias("target").merge(
        df_faturas.alias("source"),
        "target.fatura_id = source.fatura_id"  # chave única
    ).whenMatchedUpdate(
        set = {
            # Atualiza todas as colunas exceto update_date
            **{col: f"source.{col}" for col in df_faturas.columns if col != "update_date"},
            # Atualiza a update_date com a data atual
            "update_date": "current_date()"
        }
    ).whenNotMatchedInsertAll() \
     .execute()
else:
    # Se não existir, cria a tabela Delta incluindo a update_date
    df_faturas.write.format("delta").save(catalog_path)